In [1]:
# Run in terminal or command prompt
# python3 -m spacy download enimport numpy as np
import pandas as pd
import numpy as np
import re, nltk, spacy, gensim# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

     |████████████████████████████████| 1.6MB 8.5MB/s 
     |████████████████████████████████| 552kB 44.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=b435ef880a1634394eb54a10a6d88ab312b887ab5a549e9b39bd37fa9aa8aec7
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=19f1a2e006295c8c0ff6682e0b48044466020c17ce2e4492ffcef066cab2b4d2
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
path = '/content/drive/My Drive/thesis part1-122000_ final_reviews for topic modeling .csv'
df = pd.read_csv(path)

#Data Cleaning

In [4]:
# Convert to list
data = df.reviews.values.tolist()
# Remove Emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub(r"\'", "", sent) for sent in data]
pprint(data[:1])

['use the app awhile get bunch really good ringtonesnotif today tri download '
 'notify tone issue download notify tone say download chang tone default '
 'notify sound full review use the app awhile get bunch really good '
 'ringtonesnotif today tri download notify tone issue download notify tone say '
 'download chang tone default notify sound way set specif app save list notify '
 'tone go chang ca nt chang back miss cometh']


#Tokenize

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['use', 'the', 'app', 'awhile', 'get', 'bunch', 'really', 'good', 'ringtonesnotif', 'today', 'tri', 'download', 'notify', 'tone', 'issue', 'download', 'notify', 'tone', 'say', 'download', 'chang', 'tone', 'default', 'notify', 'sound', 'full', 'review', 'use', 'the', 'app', 'awhile', 'get', 'bunch', 'really', 'good', 'ringtonesnotif', 'today', 'tri', 'download', 'notify', 'tone', 'issue', 'download', 'notify', 'tone', 'say', 'download', 'chang', 'tone', 'default', 'notify', 'sound', 'way', 'set', 'specif', 'app', 'save', 'list', 'notify', 'tone', 'go', 'chang', 'ca', 'nt', 'chang', 'back', 'miss', 'cometh']]


#Stemming

In [6]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [7]:
# Initialize spacy ‘en’ model, keeping only tagger component (for efficiency)
# Run in terminal: python -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])
 
# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN','VERB']) #select noun and verb
print(data_lemmatized[:2])

['use app get ringtonesnotif today download notify tone issue download notify tone say download default notify review use app get ringtonesnotif today download notify tone issue download notify tone say download default way set list tone go', 'app wallpaper drawback app get lot pour commerce use know reason phone slow app reviewthi app wallpaper drawback app get lot pour commerce use know reason phone slow app wallpaper']


# Create the Document-Word matrix

In [8]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,
# minimum reqd occurences of a word 
                             stop_words='english',             
# remove stop words
                             lowercase=True,                   
# convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  
# num chars > 3
                             # max_features=50000,             
# max number of uniq words    
)
data_vectorized = vectorizer.fit_transform(data_lemmatized)

#Build LDA model with sklearn

In [10]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=10,               # Number of topics
                                      max_iter=10,               
# Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          
# Random state
                                      batch_size=128,            
# n docs in each learning iter
                                      evaluate_every = -1,       
# compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               
# Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [11]:
# Define Search Param
# search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}
search_params = {'n_components': [10], 'learning_decay': [.9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1,
             random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0),
       iid=True, n_jobs=1,
      #  param_grid={'n_topics': [10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       param_grid={'n_topics': [10], 'learning_decay': [0.9]},

       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

GridSearchCV(cv=None, error_score='raise',
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method=None,
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=1,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                                 total_

In [12]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 10}
Best Log Likelihood Score:  -1327286.7339386623
Model Perplexity:  575.9484903196364


In [13]:
# Create Document — Topic Matrix

lda_output = best_lda_model.transform(data_vectorized)
topicnames = ['Topic' + str(i) for i in range(best_lda_model.n_components)]
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames)

# Get dominant topic for each document

# dominant_topic = np.argmax(df_document_topic.values, axis=1)
# df_document_topic['dominant_topic'] = dominant_topic

df_document_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9
0,0.00,0.12,0.00,0.00,0.00,0.00,0.00,0.85,0.00,0.00
1,0.00,0.63,0.00,0.00,0.00,0.00,0.35,0.00,0.00,0.00
2,0.00,0.06,0.14,0.00,0.78,0.00,0.00,0.00,0.00,0.00
3,0.00,0.03,0.00,0.00,0.90,0.00,0.05,0.00,0.00,0.00
4,0.00,0.12,0.00,0.00,0.00,0.00,0.00,0.85,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
122250,0.05,0.05,0.05,0.05,0.05,0.05,0.55,0.05,0.05,0.05
122251,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.55
122252,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10
122253,0.05,0.55,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05


In [ ]:
result = pd.concat([df,df_document_topic], axis=1, sort=False)

In [ ]:
result

,reviews,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9
0,use the app awhile get bunch really good ringt...,0.00,0.12,0.00,0.00,0.00,0.00,0.00,0.85,0.00,0.00
1,the pretty nice app cool winston beauti crisp ...,0.00,0.63,0.00,0.00,0.00,0.00,0.35,0.00,0.00,0.00
2,use the app year never ani issue wide variety ...,0.00,0.06,0.14,0.00,0.78,0.00,0.00,0.00,0.00,0.00
3,love choice beauti pic mani choice kind style ...,0.00,0.03,0.00,0.00,0.90,0.00,0.05,0.00,0.00,0.00
4,use the app awhile get bunch really good ringt...,0.00,0.12,0.00,0.00,0.00,0.00,0.00,0.85,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
122250,great little app,0.05,0.05,0.05,0.05,0.05,0.05,0.55,0.05,0.05,0.05
122251,great secure line,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.55
122252,fast secure,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10
122253,veri great app alway work,0.05,0.55,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05


In [ ]:
from google.colab import  drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
result.to_csv('/drive/My Drive/thesis part1-122000_ final_reviews for topic modeling_with_topic_modeling.csv')

In [20]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        # top_keyword_locs = (-topic_weights).argsort()[:n_words]
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,feature,need,start,time,help,service,work,share,datum,product,email,month,browser,veri,address
Topic 1,work,app,plea,problem,fix,phone,say,issue,time,doe,apply,use,tri,download,error
Topic 2,upeat,app,send,star,crash,time,drive,use,item,rate,photo,issue,message,week,happen
Topic 3,game,play,download,make,podcast,expert,mode,event,task,level,point,fail,time,news,card
Topic 4,video,use,app,user,option,phone,save,money,edit,make,account,recommend,text,people,number
Topic 5,watch,version,camera,translate,voice,read,purchase,look,make,message,support,time,ask,auto,review
Topic 6,use,app,love,connect,alarm,thank,job,help,year,time,internet,abl,work,think,make
Topic 7,set,map,turn,use,speed,file,upeat,function,notify,phone,app,list,check,page,limit
Topic 8,app,add,pay,want,buy,way,quality,use,wish,place,home,view,search,thing,switch
Topic 9,order,thing,receive,prove,language,calendar,record,deliver,track,date,seller,expect,platform,food,line
